In [29]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np

In [3]:
geolocator = Nominatim(user_agent="test_tk")

In [4]:
os.makedirs('./results', exist_ok=True )

In [5]:
data_path = '../download_with_python/data/'
cur_dir = os.getcwd()

In [6]:
os.chdir(data_path)
files = glob.glob("*.json")
os.chdir(cur_dir)

In [7]:
crimes_dict = {}
offenders_dict = {}
env_dict = {}
i = 0

for file in files:
    name = file.split(sep="_")[0]
    if name == 'crime':
        crimes_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    elif name == 'offenders':
        offenders_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
        # drop registered as homeless offenders
        offenders_dict[name+f'_{i}'] = offenders_dict[name+f'_{i}'][~(offenders_dict[name+f'_{i}']['block'] == '(registered as homeless)')]
        # replace anonymized house number with 0 (replaces X with 0)
        offenders_dict[name+f'_{i}']['block'] = offenders_dict[name+f'_{i}']['block'].str.replace('X','0') + ' CHICAGO'
        offenders_dict[name+f'_{i}'][['stnum','restofadd']] = offenders_dict[name+f'_{i}']['block'].str.split(" ", n=1, expand=True)
        offenders_dict[name+f'_{i}']['stnum'] = offenders_dict[name+f'_{i}']['stnum'].astype(int) 
        offenders_dict[name+f'_{i}']['fixed_addr'] = offenders_dict[name+f'_{i}']['stnum'].astype(str) + ' ' + offenders_dict[name+f'_{i}']['restofadd']
    elif name == 'envcomplaint':
        env_dict[name+f'_{i}'] = pd.read_json((data_path+file), lines=True)
    else:
        print(f'error on {file}')


In [8]:
# TO DO
# make merger function to merge like for like

In [9]:
# define geojsons
with open('../../geojsons/Boundaries - Community Areas (current).geojson') as f:
    commarea = json.load(f)

In [10]:
def merge_df(dictionary):
    merged_df = pd.DataFrame()
    for datfram in dictionary:
        merged_df= pd.concat([merged_df,dictionary[datfram]],axis=0)
    return merged_df

In [11]:
merged_crimes = merge_df(crimes_dict)
merged_offenders = merge_df(offenders_dict)
merged_envcomplaints = merge_df(env_dict)

In [12]:
def get_commarea_env(dataframe):
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe['location.coordinates'][row][0],dataframe['location.coordinates'][row][1])
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe['community_area'][row] = feature['properties']['area_numbe']

In [13]:
qq = geolocator.geocode('7200 S SOUTH SHORE DR CHICAGO')


In [14]:
latlong = [qq.latitude, qq.longitude]
point = Point(latlong[1], latlong[0])
for feature in commarea['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
        print( feature['properties']['area_numbe'])
        

43


In [15]:
feature['properties']['area_numbe']

'9'

In [16]:
def getpoint(address):
    loc = geolocator.geocode(address)
    latlong = [loc.latitude, loc.longitude]
    point = Point(latlong[1], latlong[0])   
    return point

In [17]:
def findarea(point):    
    for feature in commarea['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
           return feature['properties']['area_numbe']
        else:
            continue
        

In [18]:
def get_commarea_offenders(dataframe):
    dataframe['community_area']=''
    i=0
    for row in dataframe.index:
        try:
            point = getpoint(dataframe['fixed_addr'][row])
            print(point)
            dataframe['community_area'][row] = findarea(point)
        except:
            dataframe['community_area'][row] = 'no match to osm'
        print(f'{round((i/dataframe.shape[0]),4)*100}% done')
        i+=1
    return dataframe


In [19]:
get_commarea_env(merged_envcomplaints)


C:\Users\tkkim\AppData\Local\Temp\ipykernel_30644\2635954889.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = feature['properties']['area_numbe']


In [20]:
merged_offenders = get_commarea_offenders(merged_offenders)


POINT (-87.56251724291809 41.765387849999996)
0.0% done


C:\Users\tkkim\AppData\Local\Temp\ipykernel_30644\509318900.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = findarea(point)


POINT (-87.6216577 41.7313634)
0.03% done
POINT (-87.65521903106898 41.963758600000006)
0.06999999999999999% done
POINT (-87.650505 41.783386)
0.1% done


C:\Users\tkkim\AppData\Local\Temp\ipykernel_30644\509318900.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['community_area'][row] = 'no match to osm'


0.13999999999999999% done
0.16999999999999998% done
POINT (-87.7313183 41.7401818)
0.21% done
POINT (-87.7873384 41.9089918)
0.24% done
0.27999999999999997% done
POINT (-87.614074 41.761955)
0.31% done
0.35000000000000003% done
POINT (-87.55550794405738 41.7481089)
0.38% done
0.42% done
0.44999999999999996% done
POINT (-87.629111 41.687146)
0.49% done
POINT (-87.6661002 41.9360873)
0.52% done
0.5599999999999999% done
POINT (-87.58834030991395 41.76535305)
0.59% done
POINT (-87.605241 41.747502)
0.63% done
POINT (-87.8095717 41.970857)
0.66% done
POINT (-87.62872998416722 41.712415899999996)
0.7000000000000001% done
POINT (-87.7057070603027 41.82551425)
0.73% done
POINT (-87.55538929949273 41.740852849999996)
0.77% done
POINT (-87.71687 41.959388)
0.8% done
POINT (-87.71575441530507 41.96133385)
0.84% done
POINT (-87.6676538 41.7123745)
0.8699999999999999% done
POINT (-87.6425 41.672427)
0.91% done
POINT (-87.72415 41.778721)
0.9400000000000001% done
0.97% done
POINT (-87.780639 41.9215

In [21]:
merged_crimes.to_csv('./results/crimes_cleaned.csv', index=False)
merged_offenders.to_csv('./results/offeders_cleaned.csv', index=False)
merged_envcomplaints.to_csv('./results/crimes_envcomplaints.csv', index=False)

In [22]:
grouped_offenders_by_commarea = merged_offenders.groupby('community_area').agg(total_offenders_in_ca=('last','count')).reset_index()
grouped_offenders_by_commarea.to_csv('./results/offenders_by_community_area.csv', index=False)

In [23]:
grouped_complaints_by_commarea = merged_envcomplaints.groupby('community_area').agg(total_complaints_in_ca=('complaint_id','count')).reset_index()
grouped_complaints_by_commarea.to_csv('./results/envcomplaints_by_community_area.csv', index=False)

In [24]:
grouped_offenders_by_commarea.head()

,community_area,total_offenders_in_ca
0,1,21
1,10,15
2,11,14
3,12,6
4,13,3


In [25]:
merged_crimes.dtypes

id                                 int64
case_number                       object
date                      datetime64[ns]
block                             object
iucr                              object
primary_type                      object
description                       object
location_description              object
arrest                              bool
domestic                            bool
beat                               int64
district                           int64
ward                               int64
community_area                     int64
fbi_code                          object
x_coordinate                     float64
y_coordinate                     float64
year                               int64
updated_on                        object
latitude                         float64
longitude                        float64
location.latitude                float64
location.longitude               float64
location.human_address            object
dtype: object

In [34]:
grouped_offenders_by_commarea['community_area'] = grouped_offenders_by_commarea['community_area'].replace('no match to osm', np.NaN)
grouped_offenders_by_commarea


,community_area,total_offenders_in_ca
0,NaN,21
1,NaN,15
2,NaN,14
3,NaN,6
4,NaN,3
...,...,...
71,NaN,11
72,NaN,24
73,NaN,16
74,NaN,4


In [35]:
big_df = merged_crimes.merge(grouped_offenders_by_commarea, how='left', on='community_area')

In [38]:
grouped_complaints_by_commarea.value_counts()

community_area  total_complaints_in_ca
1               2                         1
56              3                         1
41              1                         1
42              1                         1
43              2                         1
46              1                         1
48              4                         1
49              1                         1
5               6                         1
53              1                         1
59              2                         1
11              4                         1
6               3                         1
61              2                         1
65              1                         1
68              3                         1
7               6                         1
72              1                         1
73              1                         1
77              2                         1
4               5                         1
39              1                    

In [47]:
grouped_complaints_by_commarea['community_area'] = grouped_complaints_by_commarea['community_area'].astype('int64')

In [49]:
big_df = big_df.merge(grouped_complaints_by_commarea, how='left', on='community_area')

In [50]:
big_df.to_csv('./results/final_df.csv', index=False)